В данном файле представлены результаты 17 экспериментов ждя различных строковых ядер на разных входгых данных.
Основное ядро - LocalAlignmentStringKernel, считает все возможные локальные выравнивания между двумя последовательностями.
Для сравнения работы были взяты ядра OligoStringKernel, GaussianMatchStringKernel

In [41]:
print("*Results with random4000.fa ", "PR", "ROC ", sep="     ")
print("for enh_es_fb                ", "%.4f" %PR_result9.get_mean(), "%.5f" %ROC_result10.get_mean(), sep="  ")
print("for enh_es_800               ", "%.4f" %PR_result6.get_mean(), "%.5f" %ROC_result7.get_mean(), sep="  ")
print("for enh_es_200               ", "%.4f" %PR_result2.get_mean(), "%.5f" %ROC_result1.get_mean(), sep="  ")
print("for enh_es_mb                ", "%.4f" %PR_result4.get_mean(), "%.5f" %ROC_result3.get_mean(), sep="  ")
print("*Results with random5240.fa ", "PR", "ROC ", sep="     ")
print("for enh_es_800               ", "%.4f" %PR_result11.get_mean(), "%.5f" %ROC_result12.get_mean(), sep="  ")
print("for enh_es_200               ", "%.4f" %PR_result13.get_mean(), "%.5f" %ROC_result14.get_mean(), sep="  ")
print("for enh_es_fb                ", "%.4f" %PR_result15.get_mean(), "%.5f" %ROC_result16.get_mean(), sep="  ")
print("for enh_es_mb                ", "%.4f" %PR_result17.get_mean(), "%.5f" %ROC_result18.get_mean(), sep="  ")
print("*Results with random12000.fa", "PR", "ROC ", sep="     ")
print("for enh_es_fb                ", "%.4f" %PR_result21.get_mean(), "%.5f" %ROC_result22.get_mean(), sep="  ")
print("for enh_es_200               ", "%.4f" %PR_result23.get_mean(), "%.5f" %ROC_result24.get_mean(), sep="  ")
print("for enh_es_800               ", "%.4f" %PR_result25.get_mean(), "%.5f" %ROC_result26.get_mean(), sep="  ")
print("for enh_neuact_200           ", "%.4f" %PR_result27.get_mean(), "%.5f" %ROC_result28.get_mean(), sep="  ")
print("for enh_mb                   ", "%.4f" %PR_result9.get_mean(), "%.5f" %ROC_result9.get_mean(), sep="  ")

*Results with random4000.fa      PR     ROC 
for enh_es_fb                  0.7401  0.70571
for enh_es_800                 0.9877  1.00000
for enh_es_200                 0.9865  0.99941
for enh_es_mb                  0.9106  0.87611
*Results with random5240.fa      PR     ROC 
for enh_es_800                 0.9850  1.00000
for enh_es_200                 0.9428  0.97043
for enh_es_fb                  0.9849  0.99991
for enh_es_mb                  0.9848  0.99991
*Results with random12000.fa     PR     ROC 
for enh_es_fb                  0.9904  1.00000
for enh_es_200                 0.9296  0.96891
for enh_es_800                 0.9904  1.00000
for enh_neuact_200             0.6905  0.73083
for enh_mb                     0.7401  0.70627


In [11]:
print("Different kernels with random12000 as positives and enh_es_200 ans negatives:")
print("Kernel type            ", "PR", "ROC", sep = "         ")
print("LocalAlignmentStringKernel ", "%.4f" %PR_result23.get_mean(), "%.5f" %ROC_result24.get_mean(), sep = "     ")
print("OligoStringKernel          ", "%.4f" %PR_result_diff1.get_mean(), "%.5f" %ROC_result_diff2.get_mean(), sep = "     " )
print("GaussianMatchStringKernel  ", "%.4f" %PR_result_diff7.get_mean(), "%.5f" %ROC_result_diff8.get_mean(), sep = "     " )

Different kernels with random12000 as positives and enh_es_200 ans negatives:
Kernel type                     PR         ROC
LocalAlignmentStringKernel      0.9380     0.97009
OligoStringKernel               0.9433     0.96943
GaussianMatchStringKernel       0.5377     0.54570


Далее следует 34 ячейки с очень схожим кодом, изменения во входных данных и переменных для получения ответа.

In [1]:
from array import *
import shogun
import numpy
from shogun import LinearStringKernel
from shogun import WeightedDegreeStringKernel, SpectrumMismatchRBFKernel, WeightedDegreePositionStringKernel, GaussianKernel, LocalAlignmentStringKernel
from shogun import StringWordFeatures, StringUlongFeatures, StringCharFeatures, DNA, BinaryLabels
from shogun import PRCEvaluation, ROCEvaluation, LibSVM, MSG_INFO, KernelMachine, CSVM, IdentityKernelNormalizer
from shogun import CrossValidation, CrossValidationResult, StratifiedCrossValidationSplitting

def read_fastafile(filename):
    """
    Fasta file containing whole genome must
    have chomosomes ID as fastaID names
    """
    id = ''
    ids = []
    seqs = []
    seq = []
    minimal = 1000000000000000000
    with open(filename, 'r') as fh:
        for line in fh:
            if line[0] == '>':
                regionID=line[1:].rstrip('\n')
                regionID = regionID.split(":")[0]
                ids.append(regionID)
                if seq != []: seqs.append("".join(seq))
                seq = []
            else:
                seq.append(line.rstrip('\n').upper())
                if (len(line.rstrip('\n').upper()) < minimal):
                    minimal = len(line.rstrip('\n').upper())
        if seq != []:
            seqs.append("".join(seq))
                
    return seqs, ids, minimal


g_revcomp_mapping = []
g_kmers_list = []

def generate_kmers_list(kmerlen):
    nts = ['A', 'C', 'G', 'T']
    kmers = []
    kmers.append('')
    l = 0
    while l < kmerlen:
        imers = []
        for imer in kmers:
            for nt in nts:
                imers.append(imer + nt)
        kmers = imers
        l += 1

    return kmers


def generate_revcomp_mapping_table(kmerlen):
    kmers = g_kmers_list

    rc = {'A':'T', 'G':'C', 'C':'G', 'T':'A'}
    revcomp = lambda s: ''.join([rc[s[i]] for i in range(len(s) - 1, -1, -1)])

    kmer_id_dict = {}
    for i in range(len(kmers)):
        kmer_id_dict[kmers[i]] = i

    revcomp_mapping_table = []
    for kmerid in range(len(kmers)):
        rc_id = kmer_id_dict[revcomp(kmers[kmerid])]
        if rc_id < kmerid:
            revcomp_mapping_table.append(rc_id)
        else:
            revcomp_mapping_table.append(kmerid)

    return revcomp_mapping_table



negatives1, ids2, minimal2 = read_fastafile('enh_es_200.fa')
positives1, ids1, minimal1 = read_fastafile('random4000.fa')
cut = min(minimal1, minimal2)
seqs = positives1 + negatives1

labels = [numpy.double(1)] * len(positives1) + [numpy.double(-1)] * len(negatives1)
g_kmers_list = generate_kmers_list(6)
g_revcomp_mapping = generate_revcomp_mapping_table(6)
feats_tr = StringCharFeatures(DNA)
feats_tr.set_features(seqs)
kernel = LocalAlignmentStringKernel(feats_tr, feats_tr)

normaliz = IdentityKernelNormalizer()
normaliz.normalize(0, 0, 250)
kernel.set_normalizer(normaliz)
splitting_strategy=StratifiedCrossValidationSplitting(BinaryLabels(numpy.array(labels)), 3)
evaluation_criterium2 = ROCEvaluation()
svmC = 1.0
classifier_SVM1 = LibSVM(svmC, kernel, BinaryLabels(numpy.array(labels)))
cross_validation2 = CrossValidation(classifier_SVM1, feats_tr, BinaryLabels(numpy.array(labels)),splitting_strategy, evaluation_criterium2, False)

results2 = cross_validation2.evaluate()

ROC_result1 = CrossValidationResult.obtain_from_generic(results2)
print("mean area under the ROC curve", ROC_result1.get_mean())

mean area under the ROC curve 0.9993646739240609


In [8]:
negatives1, ids2, minimal2 = read_fastafile('enh_es_200.fa')
positives1, ids1, minimal1 = read_fastafile('random4000.fa')
cut = min(minimal1, minimal2)
seqs = positives1 + negatives1

labels = [numpy.double(1)] * len(positives1) + [numpy.double(-1)] * len(negatives1)
g_kmers_list = generate_kmers_list(6)
g_revcomp_mapping = generate_revcomp_mapping_table(6)
feats_tr = StringCharFeatures(DNA)
feats_tr.set_features(seqs)
kernel = LocalAlignmentStringKernel(feats_tr, feats_tr)

normaliz = IdentityKernelNormalizer()
normaliz.normalize(0, 0, 250)
kernel.set_normalizer(normaliz)
splitting_strategy=StratifiedCrossValidationSplitting(BinaryLabels(numpy.array(labels)), 3)
evaluation_criterium1 = PRCEvaluation()
svmC = 1.0
classifier_SVM1 = LibSVM(svmC, kernel, BinaryLabels(numpy.array(labels)))
cross_validation1 = CrossValidation(classifier_SVM1, feats_tr, BinaryLabels(numpy.array(labels)),splitting_strategy, evaluation_criterium1, False)

results1 = cross_validation1.evaluate()

PR_result2 = CrossValidationResult.obtain_from_generic(results1)
print("mean area under the PR curve", PR_result2.get_mean())

mean area under the PR curve 0.9864718220889416


In [10]:
negatives1, ids2, minimal2 = read_fastafile('enh_es_800.fa')
positives1, ids1, minimal1 = read_fastafile('random4000.fa')
cut = min(minimal1, minimal2)
seqs = positives1 + negatives1

labels = [numpy.double(1)] * len(positives1) + [numpy.double(-1)] * len(negatives1)
g_kmers_list = generate_kmers_list(6)
g_revcomp_mapping = generate_revcomp_mapping_table(6)
feats_tr = StringCharFeatures(DNA)
feats_tr.set_features(seqs)
kernel = LocalAlignmentStringKernel(feats_tr, feats_tr)

normaliz = IdentityKernelNormalizer()
normaliz.normalize(0, 0, 250)
kernel.set_normalizer(normaliz)
splitting_strategy=StratifiedCrossValidationSplitting(BinaryLabels(numpy.array(labels)), 3)
evaluation_criterium2 = ROCEvaluation()
svmC = 1.0
classifier_SVM = LibSVM(svmC, kernel, BinaryLabels(numpy.array(labels)))
cross_validation3 = CrossValidation(classifier_SVM, feats_tr, BinaryLabels(numpy.array(labels)),splitting_strategy, evaluation_criterium2, False)

results3 = cross_validation3.evaluate()

ROC_result3 = CrossValidationResult.obtain_from_generic(results3)
print("mean area under the ROC curve", ROC_result3.get_mean())

mean area under the ROC curve 0.876113260856213


In [11]:
negatives1, ids2, minimal2 = read_fastafile('enh_es_800.fa')
positives1, ids1, minimal1 = read_fastafile('random4000.fa')
cut = min(minimal1, minimal2)
seqs = positives1 + negatives1

labels = [numpy.double(1)] * len(positives1) + [numpy.double(-1)] * len(negatives1)
g_kmers_list = generate_kmers_list(6)
g_revcomp_mapping = generate_revcomp_mapping_table(6)
feats_tr = StringCharFeatures(DNA)
feats_tr.set_features(seqs)
kernel = LocalAlignmentStringKernel(feats_tr, feats_tr)

normaliz = IdentityKernelNormalizer()
normaliz.normalize(0, 0, 250)
kernel.set_normalizer(normaliz)
splitting_strategy=StratifiedCrossValidationSplitting(BinaryLabels(numpy.array(labels)), 3)
evaluation_criterium1 = PRCEvaluation()
svmC = 1.0
classifier_SVM2 = LibSVM(svmC, kernel, BinaryLabels(numpy.array(labels)))
cross_validation4 = CrossValidation(classifier_SVM2, feats_tr, BinaryLabels(numpy.array(labels)),splitting_strategy, evaluation_criterium1, False)

results4 = cross_validation4.evaluate()

PR_result4 = CrossValidationResult.obtain_from_generic(results4)
print("mean area under the PR curve", PR_result4.get_mean())

mean area under the PR curve 0.9105886916868887


In [12]:
negatives1, ids2, minimal2 = read_fastafile('enh_es_800.fa')
positives1, ids1, minimal1 = read_fastafile('random4000.fa')
cut = min(minimal1, minimal2)
seqs = positives1 + negatives1

labels = [numpy.double(1)] * len(positives1) + [numpy.double(-1)] * len(negatives1)
g_kmers_list = generate_kmers_list(6)
g_revcomp_mapping = generate_revcomp_mapping_table(6)
feats_tr = StringCharFeatures(DNA)
feats_tr.set_features(seqs)
kernel = LocalAlignmentStringKernel(feats_tr, feats_tr)

normaliz = IdentityKernelNormalizer()
normaliz.normalize(0, 0, 250)
kernel.set_normalizer(normaliz)
splitting_strategy=StratifiedCrossValidationSplitting(BinaryLabels(numpy.array(labels)), 3)
evaluation_criterium2 = ROCEvaluation()
svmC = 1.0
classifier_SVM = LibSVM(svmC, kernel, BinaryLabels(numpy.array(labels)))
cross_validation5 = CrossValidation(classifier_SVM, feats_tr, BinaryLabels(numpy.array(labels)),splitting_strategy, evaluation_criterium2, False)

results5 = cross_validation5.evaluate()

ROC_result5 = CrossValidationResult.obtain_from_generic(results5)
print("mean area under the ROC curve", ROC_result5.get_mean())

mean area under the ROC curve 0.8790000614212888


In [13]:
negatives1, ids2, minimal2 = read_fastafile('enh_neuact_200.fa')
positives1, ids1, minimal1 = read_fastafile('random4000.fa')
cut = min(minimal1, minimal2)
seqs = positives1 + negatives1

labels = [numpy.double(1)] * len(positives1) + [numpy.double(-1)] * len(negatives1)
g_kmers_list = generate_kmers_list(6)
g_revcomp_mapping = generate_revcomp_mapping_table(6)
feats_tr = StringCharFeatures(DNA)
feats_tr.set_features(seqs)
kernel = LocalAlignmentStringKernel(feats_tr, feats_tr)

normaliz = IdentityKernelNormalizer()
normaliz.normalize(0, 0, 250)
kernel.set_normalizer(normaliz)
splitting_strategy=StratifiedCrossValidationSplitting(BinaryLabels(numpy.array(labels)), 3)
evaluation_criterium1 = PRCEvaluation()
svmC = 1.0
classifier_SVM = LibSVM(svmC, kernel, BinaryLabels(numpy.array(labels)))
cross_validation6 = CrossValidation(classifier_SVM, feats_tr, BinaryLabels(numpy.array(labels)),splitting_strategy, evaluation_criterium1, False)

results6 = cross_validation6.evaluate()

PR_result6 = CrossValidationResult.obtain_from_generic(results6)
print("mean area under the PR curve", PR_result6.get_mean())

mean area under the PR curve 0.9876543209876544


In [14]:
negatives1, ids2, minimal2 = read_fastafile('enh_neuact_200.fa')
positives1, ids1, minimal1 = read_fastafile('random4000.fa')
cut = min(minimal1, minimal2)
seqs = positives1 + negatives1

labels = [numpy.double(1)] * len(positives1) + [numpy.double(-1)] * len(negatives1)
g_kmers_list = generate_kmers_list(6)
g_revcomp_mapping = generate_revcomp_mapping_table(6)
feats_tr = StringCharFeatures(DNA)
feats_tr.set_features(seqs)
kernel = LocalAlignmentStringKernel(feats_tr, feats_tr)

normaliz = IdentityKernelNormalizer()
normaliz.normalize(0, 0, 250)
kernel.set_normalizer(normaliz)
splitting_strategy=StratifiedCrossValidationSplitting(BinaryLabels(numpy.array(labels)), 3)
evaluation_criterium2 = ROCEvaluation()
svmC = 1.0
classifier_SVM = LibSVM(svmC, kernel, BinaryLabels(numpy.array(labels)))
cross_validation7 = CrossValidation(classifier_SVM, feats_tr, BinaryLabels(numpy.array(labels)),splitting_strategy, evaluation_criterium2, False)

results7 = cross_validation7.evaluate()

ROC_result7 = CrossValidationResult.obtain_from_generic(results7)
print("mean area under the ROC curve", ROC_result7.get_mean())

mean area under the ROC curve 1.0


In [15]:
negatives1, ids2, minimal2 = read_fastafile('enh_fb.fa')
positives1, ids1, minimal1 = read_fastafile('random4000.fa')
cut = min(minimal1, minimal2)
seqs = positives1 + negatives1

labels = [numpy.double(1)] * len(positives1) + [numpy.double(-1)] * len(negatives1)
g_kmers_list = generate_kmers_list(6)
g_revcomp_mapping = generate_revcomp_mapping_table(6)
feats_tr = StringCharFeatures(DNA)
feats_tr.set_features(seqs)
kernel = LocalAlignmentStringKernel(feats_tr, feats_tr)

normaliz = IdentityKernelNormalizer()
normaliz.normalize(0, 0, 250)
kernel.set_normalizer(normaliz)
splitting_strategy=StratifiedCrossValidationSplitting(BinaryLabels(numpy.array(labels)), 3)
evaluation_criterium1 = PRCEvaluation()
svmC = 1.0
classifier_SVM = LibSVM(svmC, kernel, BinaryLabels(numpy.array(labels)))
cross_validation8 = CrossValidation(classifier_SVM, feats_tr, BinaryLabels(numpy.array(labels)),splitting_strategy, evaluation_criterium1, False)

results8 = cross_validation8.evaluate()

PR_result8 = CrossValidationResult.obtain_from_generic(results8)
print("mean area under the PR curve", PR_result8.get_mean())

mean area under the PR curve 0.7646228119478065


In [16]:

negatives1, ids2, minimal2 = read_fastafile('enh_fb.fa')
positives1, ids1, minimal1 = read_fastafile('random4000.fa')
cut = min(minimal1, minimal2)
seqs = positives1 + negatives1

labels = [numpy.double(1)] * len(positives1) + [numpy.double(-1)] * len(negatives1)
g_kmers_list = generate_kmers_list(6)
g_revcomp_mapping = generate_revcomp_mapping_table(6)
feats_tr = StringCharFeatures(DNA)
feats_tr.set_features(seqs)
kernel = LocalAlignmentStringKernel(feats_tr, feats_tr)

normaliz = IdentityKernelNormalizer()
normaliz.normalize(0, 0, 250)
kernel.set_normalizer(normaliz)
splitting_strategy=StratifiedCrossValidationSplitting(BinaryLabels(numpy.array(labels)), 3)
evaluation_criterium2 = ROCEvaluation()
svmC = 1.0
classifier_SVM = LibSVM(svmC, kernel, BinaryLabels(numpy.array(labels)))
cross_validation9 = CrossValidation(classifier_SVM, feats_tr, BinaryLabels(numpy.array(labels)),splitting_strategy, evaluation_criterium2, False)

results9 = cross_validation9.evaluate()

ROC_result9 = CrossValidationResult.obtain_from_generic(results9)
print("mean area under the ROC curve", ROC_result9.get_mean())

mean area under the ROC curve 0.7062731417019915


In [17]:
negatives1, ids2, minimal2 = read_fastafile('enh_mb.fa')
positives1, ids1, minimal1 = read_fastafile('random4000.fa')
cut = min(minimal1, minimal2)
seqs = positives1 + negatives1

labels = [numpy.double(1)] * len(positives1) + [numpy.double(-1)] * len(negatives1)
g_kmers_list = generate_kmers_list(6)
g_revcomp_mapping = generate_revcomp_mapping_table(6)
feats_tr = StringCharFeatures(DNA)
feats_tr.set_features(seqs)
kernel = LocalAlignmentStringKernel(feats_tr, feats_tr)

normaliz = IdentityKernelNormalizer()
normaliz.normalize(0, 0, 250)
kernel.set_normalizer(normaliz)
splitting_strategy=StratifiedCrossValidationSplitting(BinaryLabels(numpy.array(labels)), 3)
evaluation_criterium1 = PRCEvaluation()
svmC = 1.0
classifier_SVM = LibSVM(svmC, kernel, BinaryLabels(numpy.array(labels)))
cross_validation9 = CrossValidation(classifier_SVM, feats_tr, BinaryLabels(numpy.array(labels)),splitting_strategy, evaluation_criterium1, False)

results9 = cross_validation9.evaluate()

PR_result9 = CrossValidationResult.obtain_from_generic(results9)
print("mean area under the PR curve", PR_result9.get_mean())

mean area under the PR curve 0.7401003206029638


In [18]:
negatives1, ids2, minimal2 = read_fastafile('enh_mb.fa')
positives1, ids1, minimal1 = read_fastafile('random4000.fa')
cut = min(minimal1, minimal2)
seqs = positives1 + negatives1

labels = [numpy.double(1)] * len(positives1) + [numpy.double(-1)] * len(negatives1)
g_kmers_list = generate_kmers_list(6)
g_revcomp_mapping = generate_revcomp_mapping_table(6)
feats_tr = StringCharFeatures(DNA)
feats_tr.set_features(seqs)
kernel = LocalAlignmentStringKernel(feats_tr, feats_tr)

normaliz = IdentityKernelNormalizer()
normaliz.normalize(0, 0, 250)
kernel.set_normalizer(normaliz)
splitting_strategy=StratifiedCrossValidationSplitting(BinaryLabels(numpy.array(labels)), 3)
evaluation_criterium2 = ROCEvaluation()
svmC = 1.0
classifier_SVM = LibSVM(svmC, kernel, BinaryLabels(numpy.array(labels)))
cross_validation10 = CrossValidation(classifier_SVM, feats_tr, BinaryLabels(numpy.array(labels)),splitting_strategy, evaluation_criterium2, False)

results10 = cross_validation10.evaluate()

ROC_result10 = CrossValidationResult.obtain_from_generic(results10)
print("mean area under the ROC curve", ROC_result10.get_mean())

mean area under the ROC curve 0.7057085575604094


In [20]:
negatives1, ids2, minimal2 = read_fastafile('enh_es_800.fa')
positives1, ids1, minimal1 = read_fastafile('random5240.fa')
cut = min(minimal1, minimal2)
seqs = positives1 + negatives1

labels = [numpy.double(1)] * len(positives1) + [numpy.double(-1)] * len(negatives1)
g_kmers_list = generate_kmers_list(6)
g_revcomp_mapping = generate_revcomp_mapping_table(6)
feats_tr = StringCharFeatures(DNA)
feats_tr.set_features(seqs)
kernel = LocalAlignmentStringKernel(feats_tr, feats_tr)

splitting_strategy=StratifiedCrossValidationSplitting(BinaryLabels(numpy.array(labels)), 3)
evaluation_criterium1 = PRCEvaluation()
svmC = 1.0
classifier_SVM = LibSVM(svmC, kernel, BinaryLabels(numpy.array(labels)))
cross_validation11 = CrossValidation(classifier_SVM, feats_tr, BinaryLabels(numpy.array(labels)),splitting_strategy, evaluation_criterium1, False)

results11 = cross_validation11.evaluate()

PR_result11 = CrossValidationResult.obtain_from_generic(results11)
print("mean area under the PR curve", PR_result11.get_mean())

mean area under the PR curve 0.9849992461932761


In [21]:
negatives1, ids2, minimal2 = read_fastafile('enh_es_800.fa')
positives1, ids1, minimal1 = read_fastafile('random5240.fa')
cut = min(minimal1, minimal2)
seqs = positives1 + negatives1

labels = [numpy.double(1)] * len(positives1) + [numpy.double(-1)] * len(negatives1)
g_kmers_list = generate_kmers_list(6)
g_revcomp_mapping = generate_revcomp_mapping_table(6)
feats_tr = StringCharFeatures(DNA)
feats_tr.set_features(seqs)
kernel = LocalAlignmentStringKernel(feats_tr, feats_tr)

normaliz = IdentityKernelNormalizer()
normaliz.normalize(0, 0, 250)
kernel.set_normalizer(normaliz)
splitting_strategy=StratifiedCrossValidationSplitting(BinaryLabels(numpy.array(labels)), 3)
evaluation_criterium2 = ROCEvaluation()
svmC = 1.0
classifier_SVM = LibSVM(svmC, kernel, BinaryLabels(numpy.array(labels)))
cross_validation12 = CrossValidation(classifier_SVM, feats_tr, BinaryLabels(numpy.array(labels)),splitting_strategy, evaluation_criterium2, False)

results12 = cross_validation12.evaluate()

ROC_result12 = CrossValidationResult.obtain_from_generic(results12)
print("mean area under the ROC curve", ROC_result12.get_mean())

mean area under the ROC curve 1.0


In [22]:
negatives1, ids2, minimal2 = read_fastafile('enh_fb.fa')
positives1, ids1, minimal1 = read_fastafile('random5240.fa')
cut = min(minimal1, minimal2)
seqs = positives1 + negatives1

labels = [numpy.double(1)] * len(positives1) + [numpy.double(-1)] * len(negatives1)
g_kmers_list = generate_kmers_list(6)
g_revcomp_mapping = generate_revcomp_mapping_table(6)
feats_tr = StringCharFeatures(DNA)
feats_tr.set_features(seqs)
kernel = LocalAlignmentStringKernel(feats_tr, feats_tr)

normaliz = IdentityKernelNormalizer()
normaliz.normalize(0, 0, 250)
kernel.set_normalizer(normaliz)
splitting_strategy=StratifiedCrossValidationSplitting(BinaryLabels(numpy.array(labels)), 3)
evaluation_criterium1 = PRCEvaluation()
svmC = 1.0
classifier_SVM = LibSVM(svmC, kernel, BinaryLabels(numpy.array(labels)))
cross_validation15 = CrossValidation(classifier_SVM, feats_tr, BinaryLabels(numpy.array(labels)),splitting_strategy, evaluation_criterium1, False)

results15 = cross_validation15.evaluate()

PR_result15 = CrossValidationResult.obtain_from_generic(results15)
print("mean area under the PR curve", PR_result15.get_mean())
evaluation_criterium2 = ROCEvaluation()

cross_validation16 = CrossValidation(classifier_SVM, feats_tr, BinaryLabels(numpy.array(labels)),splitting_strategy, evaluation_criterium2, False)

results16 = cross_validation16.evaluate()

ROC_result16 = CrossValidationResult.obtain_from_generic(results16)
print("mean area under the ROC curve", ROC_result16.get_mean())

mean area under the PR curve 0.9849255366025117
mean area under the ROC curve 0.9999127171161736


In [23]:
negatives1, ids2, minimal2 = read_fastafile('enh_mb.fa')
positives1, ids1, minimal1 = read_fastafile('random5240.fa')
cut = min(minimal1, minimal2)
seqs = positives1 + negatives1

labels = [numpy.double(1)] * len(positives1) + [numpy.double(-1)] * len(negatives1)
g_kmers_list = generate_kmers_list(6)
g_revcomp_mapping = generate_revcomp_mapping_table(6)
feats_tr = StringCharFeatures(DNA)
feats_tr.set_features(seqs)
kernel = LocalAlignmentStringKernel(feats_tr, feats_tr)

normaliz = IdentityKernelNormalizer()
normaliz.normalize(0, 0, 250)
kernel.set_normalizer(normaliz)
splitting_strategy=StratifiedCrossValidationSplitting(BinaryLabels(numpy.array(labels)), 3)
evaluation_criterium1 = PRCEvaluation()
svmC = 1.0
classifier_SVM = LibSVM(svmC, kernel, BinaryLabels(numpy.array(labels)))
cross_validation17 = CrossValidation(classifier_SVM, feats_tr, BinaryLabels(numpy.array(labels)),splitting_strategy, evaluation_criterium1, False)

results17 = cross_validation17.evaluate()

PR_result17 = CrossValidationResult.obtain_from_generic(results17)
print("mean area under the PR curve", PR_result17.get_mean())
evaluation_criterium2 = ROCEvaluation()

cross_validation18 = CrossValidation(classifier_SVM, feats_tr, BinaryLabels(numpy.array(labels)),splitting_strategy, evaluation_criterium2, False)

results18 = cross_validation16.evaluate()

ROC_result18 = CrossValidationResult.obtain_from_generic(results18)
print("mean area under the ROC curve", ROC_result18.get_mean())

mean area under the PR curve 0.9847747579487257
mean area under the ROC curve 0.9999113946482368


In [24]:
negatives1, ids2, minimal2 = read_fastafile('enh_fb.fa')
positives1, ids1, minimal1 = read_fastafile('random12000.fa')
cut = min(minimal1, minimal2)
seqs = positives1 + negatives1

labels = [numpy.double(1)] * len(positives1) + [numpy.double(-1)] * len(negatives1)
g_kmers_list = generate_kmers_list(6)
g_revcomp_mapping = generate_revcomp_mapping_table(6)
feats_tr = StringCharFeatures(DNA)
feats_tr.set_features(seqs)
kernel = LocalAlignmentStringKernel(feats_tr, feats_tr)

normaliz = IdentityKernelNormalizer()
normaliz.normalize(0, 0, 250)
kernel.set_normalizer(normaliz)
splitting_strategy=StratifiedCrossValidationSplitting(BinaryLabels(numpy.array(labels)), 3)
evaluation_criterium1 = PRCEvaluation()
svmC = 1.0
classifier_SVM = LibSVM(svmC, kernel, BinaryLabels(numpy.array(labels)))
cross_validation21 = CrossValidation(classifier_SVM, feats_tr, BinaryLabels(numpy.array(labels)),splitting_strategy, evaluation_criterium1, False)

results21 = cross_validation21.evaluate()

PR_result21 = CrossValidationResult.obtain_from_generic(results21)
print("mean area under the PR curve", PR_result21.get_mean())
evaluation_criterium2 = ROCEvaluation()

cross_validation22 = CrossValidation(classifier_SVM, feats_tr, BinaryLabels(numpy.array(labels)),splitting_strategy, evaluation_criterium2, False)

results22 = cross_validation22.evaluate()

ROC_result22 = CrossValidationResult.obtain_from_generic(results22)
print("mean area under the ROC curve", ROC_result22.get_mean())

mean area under the PR curve 0.9904151404151403
mean area under the ROC curve 1.0


In [5]:

negatives1, ids2, minimal2 = read_fastafile('enh_es_200.fa')
positives1, ids1, minimal1 = read_fastafile('random12000.fa')
cut = min(minimal1, minimal2)
seqs = positives1 + negatives1

labels = [numpy.double(1)] * len(positives1) + [numpy.double(-1)] * len(negatives1)
g_kmers_list = generate_kmers_list(6)
g_revcomp_mapping = generate_revcomp_mapping_table(6)
feats_tr = StringCharFeatures(DNA)
feats_tr.set_features(seqs)
kernel = LocalAlignmentStringKernel(feats_tr, feats_tr)

normaliz = IdentityKernelNormalizer()
normaliz.normalize(0, 0, 250)
kernel.set_normalizer(normaliz)
splitting_strategy=StratifiedCrossValidationSplitting(BinaryLabels(numpy.array(labels)), 3)
evaluation_criterium1 = PRCEvaluation()
svmC = 1.0
classifier_SVM = LibSVM(svmC, kernel, BinaryLabels(numpy.array(labels)))
cross_validation23 = CrossValidation(classifier_SVM, feats_tr, BinaryLabels(numpy.array(labels)),splitting_strategy, evaluation_criterium1, False)

results23 = cross_validation23.evaluate()

PR_result23 = CrossValidationResult.obtain_from_generic(results23)
print("mean area under the PR curve", PR_result23.get_mean())
evaluation_criterium2 = ROCEvaluation()

cross_validation24 = CrossValidation(classifier_SVM, feats_tr, BinaryLabels(numpy.array(labels)),splitting_strategy, evaluation_criterium2, False)

results24 = cross_validation24.evaluate()

ROC_result24 = CrossValidationResult.obtain_from_generic(results24)
print("mean area under the ROC curve", ROC_result24.get_mean())

mean area under the PR curve 0.9380467932055643
mean area under the ROC curve 0.9700914347466072


In [26]:
negatives1, ids2, minimal2 = read_fastafile('enh_es_800.fa')
positives1, ids1, minimal1 = read_fastafile('random12000.fa')
cut = min(minimal1, minimal2)
seqs = positives1 + negatives1

labels = [numpy.double(1)] * len(positives1) + [numpy.double(-1)] * len(negatives1)
g_kmers_list = generate_kmers_list(6)
g_revcomp_mapping = generate_revcomp_mapping_table(6)
feats_tr = StringCharFeatures(DNA)
feats_tr.set_features(seqs)
kernel = LocalAlignmentStringKernel(feats_tr, feats_tr)

normaliz = IdentityKernelNormalizer()
normaliz.normalize(0, 0, 250)
kernel.set_normalizer(normaliz)
splitting_strategy=StratifiedCrossValidationSplitting(BinaryLabels(numpy.array(labels)), 3)
evaluation_criterium1 = PRCEvaluation()
svmC = 1.0
classifier_SVM = LibSVM(svmC, kernel, BinaryLabels(numpy.array(labels)))
cross_validation25 = CrossValidation(classifier_SVM, feats_tr, BinaryLabels(numpy.array(labels)),splitting_strategy, evaluation_criterium1, False)

results25 = cross_validation25.evaluate()

PR_result25 = CrossValidationResult.obtain_from_generic(results25)
print("mean area under the PR curve", PR_result25.get_mean())
evaluation_criterium2 = ROCEvaluation()

cross_validation26 = CrossValidation(classifier_SVM, feats_tr, BinaryLabels(numpy.array(labels)),splitting_strategy, evaluation_criterium2, False)

results26 = cross_validation26.evaluate()

ROC_result26 = CrossValidationResult.obtain_from_generic(results26)
print("mean area under the ROC curve", ROC_result26.get_mean())

mean area under the PR curve 0.9904151404151403
mean area under the ROC curve 1.0


In [27]:

negatives1, ids2, minimal2 = read_fastafile('enh_neuact_200.fa')
positives1, ids1, minimal1 = read_fastafile('random12000.fa')
cut = min(minimal1, minimal2)
seqs = positives1 + negatives1

labels = [numpy.double(1)] * len(positives1) + [numpy.double(-1)] * len(negatives1)
g_kmers_list = generate_kmers_list(6)
g_revcomp_mapping = generate_revcomp_mapping_table(6)
feats_tr = StringCharFeatures(DNA)
feats_tr.set_features(seqs)
kernel = LocalAlignmentStringKernel(feats_tr, feats_tr)

normaliz = IdentityKernelNormalizer()
normaliz.normalize(0, 0, 250)
kernel.set_normalizer(normaliz)
splitting_strategy=StratifiedCrossValidationSplitting(BinaryLabels(numpy.array(labels)), 3)
evaluation_criterium1 = PRCEvaluation()
svmC = 1.0
classifier_SVM = LibSVM(svmC, kernel, BinaryLabels(numpy.array(labels)))
cross_validation27 = CrossValidation(classifier_SVM, feats_tr, BinaryLabels(numpy.array(labels)),splitting_strategy, evaluation_criterium1, False)

results27 = cross_validation27.evaluate()

PR_result27 = CrossValidationResult.obtain_from_generic(results27)
print("mean area under the PR curve", PR_result27.get_mean())
evaluation_criterium2 = ROCEvaluation()

cross_validation28 = CrossValidation(classifier_SVM, feats_tr, BinaryLabels(numpy.array(labels)),splitting_strategy, evaluation_criterium2, False)

results28 = cross_validation28.evaluate()

ROC_result28 = CrossValidationResult.obtain_from_generic(results28)
print("mean area under the ROC curve", ROC_result28.get_mean())

mean area under the PR curve 0.6905081495412762
mean area under the ROC curve 0.7308256613398189


In [2]:

negatives1, ids2, minimal2 = read_fastafile('enh_neuact_200.fa')
positives1, ids1, minimal1 = read_fastafile('random5240.fa')
cut = min(minimal1, minimal2)
seqs = positives1 + negatives1

labels = [numpy.double(1)] * len(positives1) + [numpy.double(-1)] * len(negatives1)
g_kmers_list = generate_kmers_list(6)
g_revcomp_mapping = generate_revcomp_mapping_table(6)
feats_tr = StringCharFeatures(DNA)
feats_tr.set_features(seqs)
kernel = LocalAlignmentStringKernel(feats_tr, feats_tr)

normaliz = IdentityKernelNormalizer()
normaliz.normalize(0, 0, 250)
kernel.set_normalizer(normaliz)
splitting_strategy=StratifiedCrossValidationSplitting(BinaryLabels(numpy.array(labels)), 3)
evaluation_criterium1 = PRCEvaluation()
svmC = 1.0
classifier_SVM = LibSVM(svmC, kernel, BinaryLabels(numpy.array(labels)))
cross_validation13 = CrossValidation(classifier_SVM, feats_tr, BinaryLabels(numpy.array(labels)),splitting_strategy, evaluation_criterium1, False)

results13 = cross_validation13.evaluate()

PR_result13 = CrossValidationResult.obtain_from_generic(results13)
print("mean area under the PR curve", PR_result13.get_mean())
evaluation_criterium2 = ROCEvaluation()

cross_validation14 = CrossValidation(classifier_SVM, feats_tr, BinaryLabels(numpy.array(labels)),splitting_strategy, evaluation_criterium2, False)

results14 = cross_validation14.evaluate()

ROC_result14 = CrossValidationResult.obtain_from_generic(results14)
print("mean area under the ROC curve", ROC_result14.get_mean())

mean area under the PR curve 0.9478373504319118
mean area under the ROC curve 0.971101903659681


In [3]:
from shogun import OligoStringKernel
negatives1, ids2, minimal2 = read_fastafile('enh_es_200.fa')
positives1, ids1, minimal1 = read_fastafile('random12000.fa')
cut = min(minimal1, minimal2)
seqs = positives1 + negatives1

labels = [numpy.double(1)] * len(positives1) + [numpy.double(-1)] * len(negatives1)
g_kmers_list = generate_kmers_list(6)
g_revcomp_mapping = generate_revcomp_mapping_table(6)
feats_tr = StringCharFeatures(DNA)
feats_tr.set_features(seqs)
kernel_new = OligoStringKernel(feats_tr, feats_tr, 7, 250)

normaliz = IdentityKernelNormalizer()
normaliz.normalize(0, 0, 250)
kernel.set_normalizer(normaliz)
splitting_strategy=StratifiedCrossValidationSplitting(BinaryLabels(numpy.array(labels)), 3)
evaluation_criterium1 = PRCEvaluation()
svmC = 1.0
classifier_SVM_diff = LibSVM(svmC, kernel_new, BinaryLabels(numpy.array(labels)))
cross_validation_diff1 = CrossValidation(classifier_SVM_diff, feats_tr, BinaryLabels(numpy.array(labels)),splitting_strategy, evaluation_criterium1, False)

results_diff1 = cross_validation_diff1.evaluate()

PR_result_diff1 = CrossValidationResult.obtain_from_generic(results_diff1)
print("mean area under the PR curve", PR_result_diff1.get_mean())
evaluation_criterium2 = ROCEvaluation()

cross_validation_diff2 = CrossValidation(classifier_SVM_diff, feats_tr, BinaryLabels(numpy.array(labels)),splitting_strategy, evaluation_criterium2, False)

results_diff2 = cross_validation_diff2.evaluate()

ROC_result_diff2 = CrossValidationResult.obtain_from_generic(results_diff2)
print("mean area under the ROC curve", ROC_result_diff2.get_mean())

mean area under the PR curve 0.9432696402593456
mean area under the ROC curve 0.9694297082228115


In [9]:
from shogun import GaussianMatchStringKernel, LinearStringKernel, PolyMatchStringKernel
negatives1, ids2, minimal2 = read_fastafile('enh_neuact_200.fa')
positives1, ids1, minimal1 = read_fastafile('random12000.fa')
cut = min(minimal1, minimal2)
seqs = positives1 + negatives1

labels = [numpy.double(1)] * len(positives1) + [numpy.double(-1)] * len(negatives1)
g_kmers_list = generate_kmers_list(6)
g_revcomp_mapping = generate_revcomp_mapping_table(6)
feats_tr = StringCharFeatures(DNA)
feats_tr.set_features(seqs)
kernel_ne = GaussianMatchStringKernel(feats_tr, feats_tr, 400)

splitting_strategy=StratifiedCrossValidationSplitting(BinaryLabels(numpy.array(labels)), 3)
evaluation_criterium1 = PRCEvaluation()
svmC = 9.0
classifier_SVM_diff7 = LibSVM(svmC, kernel_ne, BinaryLabels(numpy.array(labels)))
cross_validation_diff7 = CrossValidation(classifier_SVM_diff7, feats_tr, BinaryLabels(numpy.array(labels)),splitting_strategy, evaluation_criterium1, False)

results_diff7 = cross_validation_diff7.evaluate()

PR_result_diff7 = CrossValidationResult.obtain_from_generic(results_diff7)
print("mean area under the PR curve", PR_result_diff7.get_mean())
evaluation_criterium2 = ROCEvaluation()

cross_validation_diff8 = CrossValidation(classifier_SVM_diff7, feats_tr, BinaryLabels(numpy.array(labels)),splitting_strategy, evaluation_criterium2, False)

results_diff8 = cross_validation_diff8.evaluate()

ROC_result_diff8 = CrossValidationResult.obtain_from_generic(results_diff8)
print("mean area under the ROC curve", ROC_result_diff8.get_mean())

mean area under the PR curve 0.5376675527020516
mean area under the ROC curve 0.5456997651297203
